In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 메뉴 데이터 불러오기
file_path = 'menu.csv'  # 파일 경로에 맞게 수정
menu_df = pd.read_csv(file_path, encoding='utf-8')

# 각 건강 상태에 맞는 영양 기준을 정의
health_conditions = {
    '저염식': {'나트륨(mg)': 100, '에너지(kcal)': 500},
    '저에너지식': {'에너지(kcal)': 300, '지방(g)': 10},
    '당뇨식': {'당류(g)': 10, '탄수화물(g)': 30},
    '고단백식': {'단백질(g)': 20, '에너지(kcal)': 600}
}

# 영양 성분에서 필요한 열만 추출하고 결측치 처리
nutritional_columns = ['식품명', '나트륨(mg)', '에너지(kcal)', '지방(g)', '당류(g)', '탄수화물(g)', '단백질(g)']
menu_df = menu_df[nutritional_columns].fillna(0)

# 건강 상태에 따른 추천 함수 정의
def recommend_menu_by_health_condition(condition, menu_df, num_recommendations=5):
    condition_criteria = health_conditions.get(condition)
    
    if not condition_criteria:
        raise ValueError("올바른 건강 상태를 선택해주세요. 가능한 건강 상태: 저염식, 저에너지식, 당뇨식, 고단백식")

    # 각 메뉴가 기준에 얼마나 적합한지 평가
    menu_df['적합성 점수'] = 0  # 적합성을 위한 점수 초기화
    
    for nutrient, limit in condition_criteria.items():
        # 각 영양 성분에 대해, 기준을 초과하지 않을 경우 점수를 높임
        menu_df['적합성 점수'] += np.where(menu_df[nutrient] <= limit, 1, 0)
    
    # 적합성 점수가 높은 순서대로 추천
    recommended_menus = menu_df.sort_values(by='적합성 점수', ascending=False).head(num_recommendations)
    
    return recommended_menus[['식품명', '적합성 점수'] + list(condition_criteria.keys())]

# 예제: 저염식에 맞는 음식 추천
recommended_menus = recommend_menu_by_health_condition('저염식', menu_df)
print(recommended_menus)


         식품명  적합성 점수  나트륨(mg)  에너지(kcal)
1414  수육_소고기       2     31.0        166
988      율무죽       2      1.0         77
355     감자구이       2     97.0         76
135      기장밥       2      3.0        118
136      보리밥       2      2.0        118


In [3]:
import pandas as pd
import numpy as np
import random

# 메뉴 데이터 불러오기
file_path = 'menu.csv'  # 파일 경로에 맞게 수정
menu_df = pd.read_csv(file_path, encoding='utf-8')

# 각 건강 상태에 맞는 영양 기준을 정의
health_conditions = {
    '저염식': {'나트륨(mg)': 100, '에너지(kcal)': 500},
    '저에너지식': {'에너지(kcal)': 300, '지방(g)': 10},
    '당뇨식': {'당류(g)': 10, '탄수화물(g)': 30},
    '고단백식': {'단백질(g)': 20, '에너지(kcal)': 600}
}

# 영양 성분에서 필요한 열만 추출하고 결측치 처리
nutritional_columns = ['식품명', '나트륨(mg)', '에너지(kcal)', '지방(g)', '당류(g)', '탄수화물(g)', '단백질(g)']
menu_df = menu_df[nutritional_columns].fillna(0)

# 건강 상태에 따른 추천 함수 정의
def recommend_menu_by_health_condition(condition, menu_df, num_recommendations=5):
    condition_criteria = health_conditions.get(condition)
    
    if not condition_criteria:
        raise ValueError("올바른 건강 상태를 선택해주세요. 가능한 건강 상태: 저염식, 저에너지식, 당뇨식, 고단백식")

    # 각 메뉴가 기준에 얼마나 적합한지 평가
    menu_df['적합성 점수'] = 0  # 적합성을 위한 점수 초기화
    
    for nutrient, limit in condition_criteria.items():
        # 각 영양 성분에 대해, 기준을 초과하지 않을 경우 점수를 높임
        menu_df['적합성 점수'] += np.where(menu_df[nutrient] <= limit, 1, 0)
    
    # 적합성 점수가 높은 순서대로 정렬
    sorted_menus = menu_df.sort_values(by='적합성 점수', ascending=False)
    
    # '밥'과 '국'이 포함된 메뉴 중 상위 1개씩 무작위로 선택
    rice_menus = sorted_menus[sorted_menus['식품명'].str.contains('밥')]
    soup_menus = sorted_menus[sorted_menus['식품명'].str.contains('국')]
    
    rice_menu = rice_menus.sample(1) if not rice_menus.empty else pd.DataFrame()
    soup_menu = soup_menus.sample(1) if not soup_menus.empty else pd.DataFrame()
    
    # '밥'과 '국'을 제외한 나머지 메뉴에서 상위 추천 항목을 무작위로 선택
    remaining_menus = sorted_menus[~sorted_menus['식품명'].str.contains('밥|국')].sample(num_recommendations - 2, random_state=random.randint(0, 100))
    
    # 최종 추천 리스트 결합
    recommended_menus = pd.concat([rice_menu, soup_menu, remaining_menus]).head(num_recommendations)
    
    return recommended_menus[['식품명', '적합성 점수'] + list(condition_criteria.keys())]

# 다양한 결과를 위해 무작위 요소 추가
random.seed()  # 매번 다른 결과를 위해 시드 초기화

# 예제: 모든 건강 상태에 맞는 음식 추천
for k in health_conditions:
    recommended_menus = recommend_menu_by_health_condition(k, menu_df)
    print(f'<{k} 식단>')
    print(recommended_menus)
    print('---------------------------------------------')


<저염식 식단>
             식품명  적합성 점수  나트륨(mg)  에너지(kcal)
256          기장밥       2      2.0        121
87    미역냉국_오이_고추       1    258.0         14
780  치즈볼_고르곤 치즈볼       1    501.0        373
805  미트볼조림_샤샤삭슈카       1    293.0        108
777  떡강정_못난이소떡소떡       1    427.0        260
---------------------------------------------
<저에너지식 식단>
         식품명  적합성 점수  에너지(kcal)  지방(g)
1123   초밥_모듬       2        154   2.02
1518  묵국_메밀묵       2         22   0.40
939   매운탕_메기       2         67   1.70
1016   미음_현미       2         19   0.10
311    고구마튀김       1        233  10.90
---------------------------------------------
<당뇨식 식단>
            식품명  적합성 점수  당류(g)  탄수화물(g)
1618     볶음밥_계란       2   0.17    24.23
273   떡만두국_고기만두       2   0.19    10.70
185      오징어채볶음       2   1.50     3.43
299       청포묵무침       2   0.62     6.86
1500        닭곰탕       2   0.30     2.00
---------------------------------------------
<고단백식 식단>
             식품명  적합성 점수  단백질(g)  에너지(kcal)
1147       덮밥_참치       2    5.84   

In [4]:
for k in health_conditions:
    print(k)

저염식
저에너지식
당뇨식
고단백식


In [5]:
menu_df.columns

Index(['식품명', '나트륨(mg)', '에너지(kcal)', '지방(g)', '당류(g)', '탄수화물(g)', '단백질(g)',
       '적합성 점수'],
      dtype='object')

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 메뉴 데이터 불러오기
file_path = 'menu.csv'  # 파일 경로에 맞게 수정
menu_df = pd.read_csv(file_path, encoding='utf-8')

# 건강 상태 레이블 기준 정의
def label_health_condition(row):
    if row['나트륨(mg)'] <= 80 and row['에너지(kcal)'] <= 400:
        return '저염식'
    elif row['에너지(kcal)'] <= 250 and row['지방(g)'] <= 8:
        return '저에너지식'
    elif row['당류(g)'] <= 5 and row['탄수화물(g)'] <= 20:
        return '당뇨식'
    elif row['단백질(g)'] >= 25 and row['에너지(kcal)'] <= 500:
        return '고단백식'
    else:
        return '일반식'

# 레이블 컬럼 추가
menu_df['label'] = menu_df.apply(label_health_condition, axis=1)

# 숫자형 열만 선택하여 NaN 값을 평균으로 채우기
numeric_columns = menu_df.select_dtypes(include=[np.number]).columns
menu_df[numeric_columns] = menu_df[numeric_columns].fillna(menu_df[numeric_columns].mean())

# 학습에 사용할 피처 및 레이블 설정
features = ['나트륨(mg)', '에너지(kcal)', '지방(g)', '당류(g)', '탄수화물(g)', '단백질(g)']
X = menu_df[features]
y = menu_df['label']

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 학습
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = clf.predict(X_test)

# 평가 결과 출력
print("모델 정확도:", accuracy_score(y_test, y_pred))
print("분류 보고서:\n", classification_report(y_test, y_pred))

# 학습된 클래스(라벨) 확인
print("학습된 클래스(라벨):", clf.classes_)


모델 정확도: 0.9877675840978594
분류 보고서:
               precision    recall  f1-score   support

        고단백식       0.00      0.00      0.00         2
         당뇨식       1.00      1.00      1.00        25
         일반식       0.93      0.93      0.93        30
       저에너지식       0.99      1.00      1.00       252
         저염식       1.00      1.00      1.00        18

    accuracy                           0.99       327
   macro avg       0.79      0.79      0.79       327
weighted avg       0.98      0.99      0.98       327

학습된 클래스(라벨): ['고단백식' '당뇨식' '일반식' '저에너지식' '저염식']


/opt/anaconda3/envs/st/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/st/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/st/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
pd.concat([menu_df['식품명'],X_train, pd.DataFrame(y_pred)], axis = 1)

,식품명,나트륨(mg),에너지(kcal),지방(g),당류(g),탄수화물(g),단백질(g),0
0,상추겉절이,123.0,21.0,0.69,1.59,3.39,0.89,일반식
1,샐러드_마카로니,124.0,140.0,8.17,2.72,13.76,2.37,저에너지식
2,샐러드_옥수수,170.0,131.0,8.50,4.09,12.51,1.32,저에너지식
3,얼갈이겉절이,229.0,25.0,0.71,1.62,4.09,1.14,당뇨식
4,오이생채_오이,258.0,30.0,0.48,3.64,6.21,1.24,당뇨식
...,...,...,...,...,...,...,...,...
1627,국밥_순대국밥,126.0,75.0,2.28,0.17,10.38,3.17,NaN
1628,국밥_콩나물,172.0,52.0,0.24,0.00,10.93,1.45,NaN
1629,기장밥,1.0,166.0,0.57,0.00,36.77,3.44,NaN
1630,김밥,307.0,140.0,4.55,0.00,19.98,4.84,NaN
